# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


У нас 3214 наблюдений и 5 признаков, целевой признак is_ultra: 1 - клиент выбрал тариф ультра, 0 - клиент выбрал тариф смарт

## Разбейте данные на выборки

Выделим целевой признак

In [5]:
x = df.drop(['is_ultra'], axis=1)
y = df['is_ultra'] 

Раздели на 3 выборки

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    train_size=0.6, 
                                                    random_state=42).copy()

In [7]:
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, 
                                                    train_size=0.5, 
                                                    random_state=42).copy()

In [8]:
print(len(x_train))
print(len(x_test))
print(len(x_valid))
print(len(x_train)+len(x_valid)+len(x_test))

1928
643
643
3214


Разбиение прошло успешно

## Исследуйте модели

У нас биноминальный целевой признак, используем алгоритмы классификации.
Начнем с классификатора дерева решений:

### Классификатор дерева решений

In [9]:
best_model = None
best_result = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=42, max_depth=depth) 
    model.fit(x_train, y_train)
    predictions = model.predict(x_valid) 
    result = accuracy_score(y_valid, predictions) 
    if result > best_result:
        best_model = model
        best_result = result

In [10]:
best_model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [11]:
best_result

0.7916018662519441

Наилучший результат с точностью предстказания 79 % дает модель дерева решений с максимальной глубиной глубиной 3

### Случайный лес

In [12]:
best_model = None
best_result = 0
for est in range(1, 41):
    for depth in range (1, 13):
        model = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=depth) 
        model.fit(x_train,y_train)
        result = model.score(x_valid, y_valid) 
        if result > best_result:
            best_model = model
            best_result = result

In [13]:
best_result

0.8195956454121306

In [14]:
best_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=9, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=24,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

Наилучший результат у леса с 24 деревьями и максимальной глубиной  9

### Логистическая регрессия

In [15]:
model = LogisticRegression(random_state=42) 
model.fit(x_train,y_train)
result = model.score(x_valid, y_valid) 

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
result

0.7200622083981337

Accuracy модели 72%

## Проверьте модель на тестовой выборке

### Классификатор дерева решений

In [23]:
model = DecisionTreeClassifier(random_state=42, max_depth=3) 

In [24]:
model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [25]:
predictions = model.predict(x_test) 

In [28]:
result = accuracy_score(y_test, predictions) 
result

0.8055987558320373

На тестовой выборке дерефо решений показало точность 80,6%

### Случайный лес

In [29]:
model = RandomForestClassifier(random_state=42, n_estimators=24, max_depth=9) 

In [30]:
model.fit(x_train,y_train)    

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=9, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=24,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [32]:
result = model.score(x_test, y_test)
result

0.8164852255054432

### Логистическая регрессия

In [33]:
model = LogisticRegression(random_state=42) 

In [34]:
model.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
result = model.score(x_test, y_test)
result

0.7091757387247278

Лучший результат, как и ожидалось, показал случайный лес, accuracy 81.6%
При этом более быстрое дерево решений лишь немного уступает с accuracy 80.6%
Сильно хуже результат у модели логистической регрессии * 70,9%

## (бонус) Проверьте модели на адекватность

Попробуем создать простую модель определения тарифа:

Для начала объеденим тренировочную таблицу

In [43]:
train = x_train.join(y_train)
train

,calls,minutes,messages,mb_used,is_ultra
2369,62.0,436.40,89.0,8495.06,1
2234,70.0,489.70,13.0,10671.53,0
1058,64.0,423.95,44.0,7826.03,0
118,28.0,215.41,0.0,6045.15,0
1024,100.0,709.40,17.0,16964.33,0
...,...,...,...,...,...
1095,62.0,454.02,35.0,15018.46,0
1130,69.0,465.96,12.0,14982.27,0
1294,40.0,280.44,2.0,13934.54,0
860,72.0,410.23,68.0,16006.55,0


Сделаем группировку клиентов по использованию трафика, и проверим, какими тарифами пользуется клиенты

In [80]:
train['mb'] = pd.qcut(train['mb_used'],11)

In [81]:
train.groupby('mb')['is_ultra'].agg(['count','mean'])

,count,mean
mb,,
"(-0.001, 7850.839]",176,0.403409
"(7850.839, 10866.522]",175,0.325714
"(10866.522, 13073.669]",175,0.205714
"(13073.669, 14687.39]",175,0.205714
"(14687.39, 16290.461]",175,0.188571
"(16290.461, 17700.863]",176,0.250000
"(17700.863, 19244.023]",175,0.142857
"(19244.023, 20894.828]",175,0.228571
"(20894.828, 22821.423]",175,0.291429


Явное отсечение при использовании 27 000 мб трафика, сделаем метку:

In [82]:
train['tb'] = (train['mb_used'] < 27000)*1

In [83]:
train

,calls,minutes,messages,mb_used,is_ultra,mb,tb,min,mn,score
2369,62.0,436.40,89.0,8495.06,1,"(7850.839, 10866.522]",1,"(433.835, 526.993]",1,2
2234,70.0,489.70,13.0,10671.53,0,"(7850.839, 10866.522]",1,"(433.835, 526.993]",1,2
1058,64.0,423.95,44.0,7826.03,0,"(-0.001, 7850.839]",1,"(335.59, 433.835]",1,2
118,28.0,215.41,0.0,6045.15,0,"(-0.001, 7850.839]",1,"(-0.001, 220.612]",1,2
1024,100.0,709.40,17.0,16964.33,0,"(16290.461, 17700.863]",1,"(648.64, 1364.83]",0,1
...,...,...,...,...,...,...,...,...,...,...
1095,62.0,454.02,35.0,15018.46,0,"(14687.39, 16290.461]",1,"(433.835, 526.993]",1,2
1130,69.0,465.96,12.0,14982.27,0,"(14687.39, 16290.461]",1,"(433.835, 526.993]",1,2
1294,40.0,280.44,2.0,13934.54,0,"(13073.669, 14687.39]",1,"(220.612, 335.59]",1,2
860,72.0,410.23,68.0,16006.55,0,"(14687.39, 16290.461]",1,"(335.59, 433.835]",1,2


Тоже самое проделаем с минутами, сообщения нас не интересуют, так как из прошлого анализа, мы знаем, что этой услугой уже почти не пользуются:

In [89]:
train['min'] = pd.qcut(train['minutes'],15)

In [90]:
train.groupby('min')['is_ultra'].agg(['count','mean'])

,count,mean
min,,
"(-0.001, 93.916]",129,0.480620
"(93.916, 180.0]",128,0.265625
"(180.0, 244.532]",129,0.224806
"(244.532, 288.285]",128,0.210938
"(288.285, 335.59]",129,0.232558
"(335.59, 374.794]",128,0.203125
"(374.794, 413.492]",129,0.201550
"(413.492, 453.177]",128,0.203125
"(453.177, 489.864]",129,0.147287


In [91]:
train['mn'] = (train['minutes'] < 690)*1

In [92]:
train['score'] = train['mn'] + train['tb']

In [70]:
train

,calls,minutes,messages,mb_used,is_ultra,mb,tb,min,mn,score
2369,62.0,436.40,89.0,8495.06,1,"(-0.001, 9036.943]",1,"(433.835, 526.993]",1,2
2234,70.0,489.70,13.0,10671.53,0,"(9036.943, 12471.893]",1,"(433.835, 526.993]",1,2
1058,64.0,423.95,44.0,7826.03,0,"(-0.001, 9036.943]",1,"(335.59, 433.835]",1,2
118,28.0,215.41,0.0,6045.15,0,"(-0.001, 9036.943]",1,"(-0.001, 220.612]",1,2
1024,100.0,709.40,17.0,16964.33,0,"(14805.561, 16985.75]",1,"(648.64, 1364.83]",0,1
...,...,...,...,...,...,...,...,...,...,...
1095,62.0,454.02,35.0,15018.46,0,"(14805.561, 16985.75]",1,"(433.835, 526.993]",1,2
1130,69.0,465.96,12.0,14982.27,0,"(14805.561, 16985.75]",1,"(433.835, 526.993]",1,2
1294,40.0,280.44,2.0,13934.54,0,"(12471.893, 14805.561]",1,"(220.612, 335.59]",1,2
860,72.0,410.23,68.0,16006.55,0,"(14805.561, 16985.75]",1,"(335.59, 433.835]",1,2


In [93]:
train.groupby('score')['is_ultra'].agg(['count','mean'])

,count,mean
score,,
0,55,0.909091
1,318,0.654088
2,1555,0.215434


In [94]:
train.groupby('tb')['is_ultra'].agg(['count','mean'])

,count,mean
tb,,
0,168,0.797619
1,1760,0.260795


In [95]:
train.groupby('mn')['is_ultra'].agg(['count','mean'])

,count,mean
mn,,
0,260,0.669231
1,1668,0.251199


Наша модель предполагает, если score ниже 2, клиент пользуется тарифом ультра, проверим на тестовой выборке:

In [96]:
test = x_test.join(y_test)

In [97]:
test['mn'] = (test['minutes'] < 690)*1

In [98]:
test['tb'] = (test['mb_used'] < 27000)*1

In [101]:
test['score'] = test['mn'] + test['tb']

In [102]:
test['predictions'] = (test['score'] < 2)*1
test


,calls,minutes,messages,mb_used,is_ultra,mn,tb,score,predictions
1545,30.0,166.51,77.0,12199.43,0,1,1,2,0
3173,73.0,430.76,84.0,26117.92,0,1,1,2,0
2290,82.0,510.85,0.0,20154.23,1,1,1,2,0
2645,87.0,568.92,60.0,18257.71,0,1,1,2,0
916,50.0,375.91,35.0,12388.40,0,1,1,2,0
...,...,...,...,...,...,...,...,...,...
1651,58.0,408.53,2.0,15346.04,0,1,1,2,0
842,74.0,476.37,76.0,13424.26,0,1,1,2,0
532,75.0,496.06,14.0,13850.84,1,1,1,2,0
456,76.0,593.77,0.0,11311.21,1,1,1,2,0


In [105]:
test['accuracy'] = (test['is_ultra'] == test['predictions'])*1

In [108]:
accuracy = test['accuracy'].mean()
accuracy

0.7884914463452566

Точность модели случайного леса 81,6% выше точности модели созданной исходя из логичных предположений вручную 78,8%.
Модель адекватна и её можно использовать в для принятия решений.

## Вывод

В ходе работы мы выгрузили и изучили данные.  На входе у нас было 3214 наблюдений и 5 признаков, целевой признак is_ultra: 1 - клиент выбрал тариф ультра, 0 - клиент выбрал тариф смарт.

Разбили на 3 выборки исходя из пропорции 3:1:1, для подготовки модели и последующей проверки.

Исследовали следующие модели:
    - классификатор дерева решений 
    - случайный лес
    - логистическая регрессия
Для каждой модели нашли лучшие гиперпараметры
После провели проверку на тестовой выборке результаты каждой модели
Лучший результат, как и ожидалось, показал случайный лес, accuracy 81.6%
При этом более быстрое дерево решений лишь немного уступает с accuracy 80.6%
Сильно хуже результат у модели логистической регрессии * 70,9%

Для проверки адекватности модели мы построили ручную модель исходя из параметров использования интернета и минут.
Её результат accuracy 78.8% оказался выше результата логистической регрессии, но ниже результата модели случайный лес с найденными гиперпараметрами.